# **Predicting Stock Movements using News Sentiment Analysis & ML**


Author: Subhan Chiluwal

This project aims to predict stock price movements using news sentiment analysis and machine learning. By analyzing the sentiment of news articles related to a particular stock (in this case, Apple), the goal is to build a model that can predict whether the stock price will go up or down, and provide a recommendation to **buy**, **sell**, or **hold** the stock.

### **Features**
1. **News sentiment analysis**: Sentiment analysis is performed on news headlines to determine whether the news is positive, negative, or neutral.
2. **Stock price data**: Historical stock price data is fetched to analyze price trends.
3. **Machine learning model**: A logistic regression model is trained using sentiment scores and stock price movement data to make predictions.
4. **Trading recommendation**: Based on the model’s prediction, a trading decision is made (whether to buy, sell, or hold the stock).


### **Requirements**
To run this project, you'll need to install the following Python libraries:

- **yfinance**: For fetching stock price data from Yahoo Finance.
- **newsapi-python**: For accessing the News API to get real-time news articles.
- **vaderSentiment**: For performing sentiment analysis on the fetched news headlines.
- **pandas**: For data manipulation and analysis.
- **scikit-learn**: For building the machine learning model and evaluating its performance.

You can install these libraries by running the following command:

In [ ]:
pip install yfinance newsapi-python vaderSentiment pandas scikit-learn

### Step 1: Fetch News and Analyze Sentiment
We’ll use NewsAPI to get the latest stock-related news.This will automatically get the latest financial news about the stock.

In [2]:
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='Your API Key')

# Get latest news about Apple (AAPL)
news_articles = newsapi.get_everything(q='Apple stock', language='en', sort_by='publishedAt', page_size=10)

# Extract headlines
news_headlines = [article['title'] for article in news_articles['articles']]

for i, headline in enumerate(news_headlines):
    print(f"{i+1}. {headline}")


1. CNBC Daily Open: For nervous investors, Fed’s dot plot comes as a relief
2. Is the Trump trade over?
3. Buffett Indicator Sounds the Alarm on Stocks
4. It’s Time to Take Action—Even Though It Feels Dangerous
5. Trump Wants Credit for That Too
6. Ignore the ‘AI Gloom’ and Keep Buying Apple Stock
7. YouTube, FOIA.gov, Careless People, More: Wednesday Afternoon ResearchBuzz, March 19, 2025
8. Lectric March Madness e-bike bundles and price cuts from $999, Segway Spring Sale takes 50% off e-scooters, Aventon clearance, more
9. Exclusive—Peter Narvarro: China Is the World’s Biggest Cheater, But the EU and Vietnam Run Close Seconds


### Step 2: Analyze Sentiment
We’ll use VADER Sentiment Analysis to determine if the news is positive, negative, or neutral.

💡 Sentiment Score Meaning:

Positive Sentiment (Score > 0.05) → Good news → Stock price may increase 📈 <br>
Negative Sentiment (Score < -0.05) → Bad news → Stock price may drop 📉<br>
Neutral Sentiment (Between -0.05 and 0.05) → No significant effect

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Analyze sentiment of each headline
sentiment_scores = [analyzer.polarity_scores(headline)['compound'] for headline in news_headlines]

for i, score in enumerate(sentiment_scores):
    print(f"News {i+1}: Sentiment Score = {score}")


News 1: Sentiment Score = 0.25
News 2: Sentiment Score = 0.0
News 3: Sentiment Score = -0.34
News 4: Sentiment Score = -0.4767
News 5: Sentiment Score = 0.3818
News 6: Sentiment Score = -0.3612
News 7: Sentiment Score = -0.3612
News 8: Sentiment Score = -0.6249
News 9: Sentiment Score = -0.3071


### Step 3: Fetch Apple Stock Data
Now, let’s get Apple's stock price for last 7 days using yfinance.

In [4]:
import yfinance as yf

# Download Apple (AAPL) stock data
stock_data = yf.download('AAPL', period='7d', interval='1d')

print(stock_data.tail())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-03-14  213.490005  213.949997  209.580002  211.250000  60107600
2025-03-17  214.000000  215.220001  209.970001  213.309998  48073400
2025-03-18  212.690002  215.149994  211.490005  214.160004  42432400
2025-03-19  215.240005  218.759995  213.750000  214.220001  54385400
2025-03-20  214.100006  217.490005  212.220001  213.990005  48799300


### Step 4: Train the Machine Learning Model
Now, we’ll combine the sentiment scores with the stock price movement to train a machine learning model. This model predicts if the stock price will go up or down based on sentiment.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf

stock_symbol = "AAPL"
stock = yf.Ticker(stock_symbol)
stock_data = stock.history(period="60d")  # Get last 60 days of data

stock_data['Sentiment'] = sentiment_scores[0]  # Using the latest sentiment score for simplicity

stock_data['Price_Change'] = stock_data['Close'].pct_change().shift(-1)  # # Calculate Price Change(%)

# Define Target Variable (1 = Price goes up, 0 = Price goes down)
stock_data['Price_Up'] = (stock_data['Price_Change'] > 0).astype(int)

stock_data = stock_data.dropna()

X = stock_data[['Sentiment']]
y = stock_data['Price_Up']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Making Predictions on Test Data
y_pred = model.predict(X_test)

# Evaluating Model Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"🔍 Model Accuracy: {accuracy:.2f}")

🔍 Model Accuracy: 0.67


### Step 5: Make Trading Decisions Based on Sentiment
Now, let’s use our trained model to predict the next move and decide whether to buy or sell.

In [7]:
# Predict based on the latest sentiment score
latest_sentiment = sentiment_scores[0]  # Latest sentiment score
prediction = model.predict([[latest_sentiment]])

# Trading Decision
if prediction == 1:
    print("📈 BUY Apple Stock!")
elif prediction == 0:
    print("📉 SELL Apple Stock!")
else:
    print("⚖️ HOLD Apple Stock!")

📉 SELL Apple Stock!
